In [1]:
def get_file_infos(filename,dl_path):
    f_s=dl_path.split('/')
    suborigin_=f_s[6]
    origin_=f_s[2]
    nature_=f_s[5]
    year_=f_s[3]
    
    import datetime
    from datetime import datetime

    f_s = filename.split('_')
    date_=f_s[2]
    year_=date_[0:4]
    month_=date_[4:6]
    day_=date_[6:9]
    time_= f_s[3]

    datetime_str=date_ + '_' + time_
    timestamp_=datetime.strptime(datetime_str,"%Y%m%d_%H%M%S")
    
    return (nature_,origin_,suborigin_,year_,timestamp_)

In [2]:
from tqdm import tqdm

In [3]:
    # Import S3 connectors librairies
    #from awsPy.aws_authorization import aws_connector
    import importlib.util
    spec = importlib.util.spec_from_file_location("aws_connector.py", "../../../../aws-python/awsPy/aws_authorization/aws_connector.py")
    aws_connector = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(aws_connector)

    #from awsPy.aws_s3 import service_s3
    import importlib.util
    spec = importlib.util.spec_from_file_location("service_s3.py", "../../../../aws-python/awsPy/aws_s3/service_s3.py")
    service_s3 = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(service_s3)

    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')

In [4]:
#s3 = boto3.resource('s3')
bucket_name='calfdata'
bucket = client['resource'].Bucket(bucket_name)

In [ ]:
#Dossier source à parcourir
'''prefix_root='INPI/TC_1/Flux'# Attention pas plus loin dans l'arborescence que 'INPI/TC_1/Flux/'
# Calcul de la liste des dossiers qui devront être traités

list_folders=[]
for y in range(2017, 2018):
    for m in range(1, 13):
        m_ = '0'+ str(m) if m<10 else str(m)
        for n in ['ETS']:#['ACTES','ETS','OBS','PM','PP','REP','COMPTES_ANNUELS']
            for s in ['EVT','NEW']:
                prefix_="{}/{}/{}/{}/{}".format(prefix_root,y,m_,n,s)
                list_folders.append(prefix_)
list_folders'''

In [ ]:
# Recréation pour correction du Flux/2018/Actes/New
'''list_folders=['INPI/TC_1/Flux/2018/01/OBS/EVT',
             'INPI/TC_1/Flux/2018/02/OBS/EVT',
             'INPI/TC_1/Flux/2018/03/OBS/EVT',
              'INPI/TC_1/Flux/2018/04/OBS/EVT',
              'INPI/TC_1/Flux/2018/05/OBS/EVT',
              'INPI/TC_1/Flux/2018/06/OBS/EVT',
              'INPI/TC_1/Flux/2018/07/OBS/EVT',
              'INPI/TC_1/Flux/2018/08/OBS/EVT',
              'INPI/TC_1/Flux/2018/09/OBS/EVT',
              'INPI/TC_1/Flux/2018/10/OBS/EVT',
              'INPI/TC_1/Flux/2018/11/OBS/EVT',
              'INPI/TC_1/Flux/2018/12/OBS/EVT'
             ]'''

In [5]:
import utils as u

In [6]:
list_folders=['INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017']

In [27]:
%%capture cap_out --no-stderr
import pandas as pd
import io
from io import StringIO

log = open('data/log-flux-2017-ets.txt', 'w+')

for prefix_ in list_folders:
    print('treat ' + prefix_)
    list_bucket = list(bucket.objects.filter(Prefix=prefix_))
    list_bucket=list_bucket[99000:]
    print(str(len(list_bucket)))
    
    
    for obj in tqdm(list_bucket):
        
        try:
            key = obj.key
            body = obj.get()['Body'].read()
            print(key, file=log)

            s=key.split('/')
            filename=s[len(s)-1]

            (nature_,origin_,suborigin_,year_,timestamp_) = u.get_file_infos(filename)
            dest_path="{}/{}/{}/{}/{}".format('INPI/TC_1/01_donnee_source',origin_,year_,nature_,suborigin_)
            dest_full_path="{}/{}".format(dest_path,filename)
            
            
            
            if nature_ == 'ETS':
            
                df = pd.read_csv(io.BytesIO(body), header=0, dtype=str, sep = ';',error_bad_lines=False)
                if nature_ == 'REP':
                    # Rename col 'date_greffe' to 'Date_Greffe'
                    df=df.rename(columns={"date_greffe": "Date_Greffe"})

                df['csv_source']=filename
                df['nature']=nature_
                df['type']=origin_
                df['origin']=suborigin_

                if origin_=='Stock' and year_ == '2017' and nature_ == 'ACTES': 
                    # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
                    print('toto')
                    df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
                elif origin_=='Stock' and year_ == '2017' and nature_ == 'COMPTES_ANNUELS': 
                    # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
                    df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
                elif origin_=='Stock' and year_ == '2017': 
                    # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Greffe
                    df['file_timestamp']= df['Date_Greffe'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
                else:
                    df['file_timestamp']=timestamp_ 

                
                # Save file to destination in S3
                # Create buffer
                csv_buffer = StringIO()
                # Write dataframe to buffer
                df.to_csv(csv_buffer, sep=";", index=False)
                # Create S3 object
                s3_resource = client['resource']
                # Write buffer to S3 object
                s3_resource.Object(bucket_name, dest_full_path).put(Body=csv_buffer.getvalue())
        except Exception as e:
            print(e)
            
log.close()        

100%|██████████| 209074/209074 [3:18:49<00:00, 15.68it/s]  


In [9]:
var = cap_out.stdout
var

'treat INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017\n308074\n'

In [24]:
list_bucket[99000].key

'INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2017/07/14/9401/94/9401_94_20170714_065935_2_PM_EVT.csv'